In [ ]:
import os
import re
import csv
import json
from collections import defaultdict
import numpy as np
import networkx as nx
from tqdm import tqdm 
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression 
import warnings 
import sklearn.multiclass

# 🚨🚨🚨 환경 설정 및 경고 무시 (오류 방지) 🚨🚨🚨
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.multiclass')
warnings.filterwarnings("ignore", category=UserWarning, message='Label not .* is present in all training examples.')


# --- 🚨 파일 경로 설정 (Amazon 환경 경로 유지) ---
DATA_BASE = "/mnt/custom-file-systems/s3/shared/seonga"

TFIDF_RANKING_PATH = f"{DATA_BASE}/ranked_keywords_top10_final.csv" 
CLASS_DESCRIPTION_PATH = f"{DATA_BASE}/gpt_class_descriptions_100calls.json" 
# ============================================
# 0. Paths & Globals
# ============================================

TRAIN_CORPUS = f"{DATA_BASE}/train_corpus.txt"
TEST_CORPUS  = f"{DATA_BASE}/test_corpus.txt"

STOPWORD_PATH = f"{DATA_BASE}/stopword.txt"
CLASS_PATH    = f"{DATA_BASE}/classes.txt"
KEYWORD_PATH  = f"{DATA_BASE}/class_related_keywords.txt"
HIER_PATH     = f"{DATA_BASE}/class_hierarchy.txt"

STRONG_SILVER_PATH = f"{DATA_BASE}/2000_gpt_667call.csv"
TRAIN_SILVER_PATH  = f"{DATA_BASE}/train_silver_heuristic_01.csv"
FINAL_SUBMISSION   = f"{DATA_BASE}/result/amz_final4.csv" # 파일명 변경

TOTAL_CLASSES = 531 
SEED = 42
PCA_COMPONENTS = 256 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[SETUP] Running on device: {device}")

# 🚨 모델: all-MiniLM-L12-v2 사용
BERT_MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
bert_tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
bert_model = AutoModel.from_pretrained(BERT_MODEL_NAME).to(device)
bert_model.eval()

pca = PCA(n_components=PCA_COMPONENTS, random_state=SEED) 


# ============================================
# 1. Loaders & Utilities
# ============================================
def load_stopwords(path):
    stop = set()
    if not os.path.exists(path): return stop
    with open(path, "r", encoding="utf-8") as f:
        for w in f:
            w = w.strip().lower()
            if w: stop.add(w)
    return stop

def load_class_names(path):
    name2id = {}
    if not os.path.exists(path): return name2id
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split(maxsplit=1) 
            if len(parts) >= 2:
                cid = parts[0]; cname = parts[1]
                if cid.isdigit(): name2id[cname] = int(cid)
    return name2id

def load_keywords(path, name2id):
    kw_map = {}
    if not os.path.exists(path): return kw_map
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if ":" not in line: continue
            cname, kws = line.split(":", 1)
            cname = cname.strip(); kws = [k.strip().lower() for k in kws.split(",") if k.strip()]
            if cname in name2id: kw_map[name2id[cname]] = kws
    return kw_map

def load_hierarchy(path, name2id):
    parent_map, child_map = {}, {}
    G = nx.DiGraph()
    if not os.path.exists(path): return parent_map, child_map, G
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = re.split(r'[\s,]+', line.strip()) 
            if len(parts) >= 2:
                p, c = parts[0], parts[1]
                try:
                    pid, cid = int(p), int(c)
                    parent_map.setdefault(pid, set()).add(cid)
                    child_map.setdefault(cid, set()).add(pid)
                    G.add_edge(pid, cid)
                except ValueError: pass
    return parent_map, child_map, G

def load_corpus(path, desc="Loading Corpus"):
    pid2text = {}
    if not os.path.exists(path): print(f"Error: Corpus file not found at {path}"); return pid2text
    num_lines = sum(1 for _ in open(path, "r", encoding="utf-8"))
    with open(path, "r", encoding="utf-8") as f:
        for line in tqdm(f, total=num_lines, desc=desc):
            parts = line.strip().split("\t", 1)
            if len(parts) == 2: pid2text[parts[0]] = parts[1]
            elif len(parts) == 1: pid2text[str(len(pid2text))] = parts[0]
            
    if not any(pid.isdigit() for pid in pid2text.keys()) or ' ' in next(iter(pid2text.keys()), ''):
          return {str(i): text for i, text in enumerate(pid2text.values())}

    return pid2text

def load_silver_labels(path):
    pid2label = {}
    if not os.path.exists(path): print(f"Error: Silver label file not found at {path}"); return pid2label
    num_lines = sum(1 for _ in open(path, "r", encoding="utf-8"))
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.reader(f); header = next(reader, None)
        for row in tqdm(reader, total=num_lines-1, desc=f"Loading Labels from {os.path.basename(path)}"):
            if len(row) != 2: continue
            pid, labels = row
            labels = labels.strip()
            if not labels: pid2label[pid] = []; continue
            pid2label[pid] = [int(x) for x in labels.split(",") if x.strip().isdigit()]
    return pid2label

def load_tfidf_ranking(path):
    if not os.path.exists(path): 
        print(f"Error: TFIDF Ranking file not found at {path}"); 
        return {}
        
    df = pd.read_csv(path)
    tfidf_map = {}
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Loading TFIDF Map"):
        pid = str(row['doc_id'])
        try:
            kws_str = row['keywords'].strip('[]')
            scores_str = row['scores'].strip('[]')

            kws = [k.strip().replace("'", "").replace('"', '').replace(' ', '') for k in kws_str.split(',') if k.strip()]
            scores = [float(s.strip()) for s in scores_str.split(',') if s.strip()]
            
            if len(kws) == len(scores):
                tfidf_map[pid] = {'keywords': kws, 'scores': scores}
        except Exception as e:
            continue
            
    return tfidf_map

@torch.no_grad()
def get_bert_embeddings(text_list, tokenizer, model, device, batch_size=64, description="SBERT Embedding"): 
    all_embs = []
    text_list = [str(t) if pd.isna(t) else t for t in text_list]
    
    batches = list(range(0, len(text_list), batch_size))
    for i in tqdm(batches, desc=description):
        batch_texts = text_list[i : i + batch_size]
        
        enc = tokenizer(batch_texts, return_tensors="pt", truncation=True, 
                              max_length=512, padding=True).to(device)
        
        out = model(**enc)
        
        cls_embs = out.last_hidden_state[:, 0, :]
        all_embs.append(cls_embs.cpu())
        
    if all_embs: 
        return torch.cat(all_embs, dim=0).numpy()
    else: 
        return np.empty((0, model.config.hidden_size))
# ------------------------------------------------------------------------------------


def clean_text_to_tokens(text, stopwords):
    text = re.sub(r"[^A-Za-z0-9]+", " ", text.lower())
    return [t for t in text.split() if t not in stopwords]

# 키워드 기반 특징 벡터 (X_KEYWORD) 생성 (분류기 입력용)
def build_keyword_features(pids, pid2text, keyword_map, tfidf_ranking_map, num_classes, stopwords):
    print("\n[Feature Upgrade] Building Keyword Feature Vector (X_KEYWORD)...")
    
    X_KEYWORD = np.zeros((len(pids), num_classes), dtype=np.float32)
    pid_to_index = {pid: i for i, pid in enumerate(pids)}
    
    for pid in tqdm(pids, desc="Generating Keyword Features"):
        idx = pid_to_index[pid]
        text = pid2text.get(pid, "")
        
        tokens = clean_text_to_tokens(text, stopwords)
        token_set = set(tokens)
        
        tfidf_data = tfidf_ranking_map.get(pid)
        tfidf_kws = tfidf_data['keywords'] if tfidf_data else []
        tfidf_scores = tfidf_data['scores'] if tfidf_data else []
        
        for cid, kws in keyword_map.items():
            if cid >= num_classes: continue
            
            score = 0.0
            for kw in kws:
                if kw in token_set:
                    score += 1.0
                    
                    try:
                        kw_index = tfidf_kws.index(kw)
                        score += tfidf_scores[kw_index] * 0.5 
                    except ValueError:
                        pass
            
            X_KEYWORD[idx, cid] = score
            
    max_val = np.max(X_KEYWORD, axis=0, keepdims=True)
    max_val[max_val == 0] = 1.0 
    X_KEYWORD = X_KEYWORD / max_val
    
    return X_KEYWORD


# ----------------------------------------------------
# TaxoClass 지식 로드 및 특징 구축 함수 (키워드 통합)
# ----------------------------------------------------

def load_class_descriptions(path, id_to_name, keyword_map):
    """ GPT로 생성된 클래스 설명을 로드하고, 키워드 정보를 통합합니다. """
    
    descriptions = id_to_name.copy()
    
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            
            loaded_descriptions = {
                int(k): v
                for k, v in data.items() 
                if isinstance(v, str) and v.strip()
            }
            
            for cid, name in id_to_name.items():
                if cid in loaded_descriptions:
                    descriptions[cid] = loaded_descriptions[cid]
        except Exception as e:
            print(f"Warning: Could not load GPT descriptions. Using default names. Error: {e}")
            pass
            
    final_map = {}
    for cid, text in descriptions.items():
        keyword_list = keyword_map.get(cid)
        
        if keyword_list:
            keywords_str = ", ".join(keyword_list)
            cleaned_text = text.rstrip('.')
            final_map[cid] = f"{cleaned_text}. Keywords: {keywords_str}."
        else:
            final_map[cid] = text
            
    return final_map


@torch.no_grad()
def build_class_features(id_to_description, bert_tokenizer, bert_model, device, G, num_classes, pca_model):
    """
    TaxoClass의 클래스 특징 벡터 (Description + Contextual)를 구축합니다.
    """
    class_texts = ["" for _ in range(num_classes)]
    for cid, text in id_to_description.items():
        if cid < num_classes:
            class_texts[cid] = text

    print("\n[TaxoClass] 1/3: Generating Base Embeddings from Descriptions...")
    base_embeddings = get_bert_embeddings(class_texts, bert_tokenizer, bert_model, device, 
                                             batch_size=64, description="Encoding Class Descriptions")
    
    print("[TaxoClass] 2/3: Applying Contextual Embedding (Hierarchy Smoothing)...")
    context_embeddings = base_embeddings.copy()
    
    try:
        for node in reversed(list(nx.topological_sort(G))):
            if node >= num_classes: continue
            
            for parent in G.predecessors(node):
                if parent < num_classes:
                    context_embeddings[parent] += base_embeddings[node] * 0.2
                    context_embeddings[parent] /= 1.2 

    except nx.NetworkXUnfeasible:
        print("WARNING: Hierarchy Graph has cycles, skipping contextual smoothing.")

    print("[TaxoClass] 3/3: Applying PCA Transform...")
    X_CLASS_FEATURES = pca_model.transform(context_embeddings)
    
    return X_CLASS_FEATURES


# ----------------------------------------------------
# GNN 개념 적용 함수 (Score Refinement)
# ----------------------------------------------------
def apply_gnn_score_refinement(scores_matrix, G, N_classes):
    refined_scores = scores_matrix.copy()
    
    try:
        for node in reversed(list(nx.topological_sort(G))):
            if node >= N_classes: continue
            
            for parent in G.predecessors(node):
                if parent < N_classes:
                    refined_scores[:, parent] = np.maximum(
                        refined_scores[:, parent], 
                        refined_scores[:, node]
                    )
    except nx.NetworkXUnfeasible:
        pass

    return refined_scores

# ----------------------------------------------------
# 휴리스틱 점수 계산 함수 (TFIDF, Keyword) - 최종 융합용
# ----------------------------------------------------
def score_text_with_tfidf(
    pid, tokens, keyword_map, parent_map, child_map, tfidf_ranking_map,
    w_exact=1.0, w_tfidf_boost=0.5, w_emotion=0.0, w_hier_child=0.5
):
    """ TFIDF 및 키워드 매칭 기반의 독립적인 휴리스틱 점수 계산 """
    scores = defaultdict(float); token_set = set(tokens)
    
    for cid, kws in keyword_map.items():
        for kw in kws:
            if kw in token_set:
                scores[cid] += w_exact

    if pid in tfidf_ranking_map:
        tfidf_data = tfidf_ranking_map[pid]
        tfidf_kws = tfidf_data['keywords']
        tfidf_scores = tfidf_data['scores']
        
        for kw, score in zip(tfidf_kws, tfidf_scores):
            for cid, kws in keyword_map.items():
                if kw in kws:
                    scores[cid] += w_tfidf_boost * score

    # 계층 구조 전파 (자식 -> 부모)
    for cid, sc in list(scores.items()):
        if sc <= 0: continue
        if cid in child_map:
            for p in child_map[cid]: scores[p] += sc * w_hier_child 

    return scores


# ----------------------------------------------------
# 🚨 새 함수: 최소 레이블 개수 제약 기반 정제
# ----------------------------------------------------
def refine_labels_with_min_constraint(sorted_scores, confidence_threshold, min_labels=2, max_labels=3):
    """
    최종 예측 점수 리스트를 받아, 최소 2개, 최대 3개의 레이블을 선택합니다.
    - 2개 미만이 임계값을 넘으면, 상위 2개는 강제로 포함합니다.
    """
    selected = []
    
    # 1. 임계값을 넘는 레이블 선택 (최대 3개)
    for cid, sc in sorted_scores:
        if len(selected) >= max_labels: break
        if sc >= confidence_threshold:
            selected.append(cid)
    
    # 2. 최소 개수 제약 조건 적용 (최소 2개)
    if len(selected) < min_labels:
        top_k_candidates = sorted_scores[:max_labels]
        
        forced_selection = []
        for i, (cid, sc) in enumerate(top_k_candidates):
            if len(forced_selection) >= min_labels: break
            # 점수가 0이 아닌 유효한 클래스만 고려
            if sc > 1e-6 and cid not in forced_selection: 
                 forced_selection.append(cid)
        
        final_selection = list(set(selected + forced_selection))
        
        # 점수 순으로 다시 정렬 (중요)
        score_dict = {cid: sc for cid, sc in sorted_scores}
        final_selection.sort(key=lambda cid: score_dict.get(cid, 0.0), reverse=True)
        
        selected = final_selection[:max_labels]
        
    if not selected:
        selected = [0]
        
    return selected

# ----------------------------------------------------
# 🚨 새 함수: 계층 구조 일관성 확보 로직 (Silver Label 수정용)
# ----------------------------------------------------
def enforce_hierarchical_consistency(
    selected_labels, parent_map, score_dict, max_labels
):
    """
    선택된 레이블 집합에 대해 계층적 일관성을 확보하고, 레이블 수가 초과되면 점수가 낮은 레이블을 제거합니다.
    """
    # 1. 일관성 확보를 위해 추가해야 할 부모 노드 찾기
    labels_to_add = set()
    current_labels = set(selected_labels)
    
    for cid in selected_labels:
        # cid의 부모들을 확인합니다.
        for parent_cid in parent_map.get(cid, set()):
            # 부모가 현재 레이블 집합에 없으면 추가 목록에 넣습니다.
            if parent_cid not in current_labels:
                labels_to_add.add(parent_cid)

    if not labels_to_add:
        return selected_labels
        
    # 2. 부모 노드 추가
    final_labels_set = current_labels.union(labels_to_add)
    final_labels_list = list(final_labels_set)

    # 3. 레이블 수가 max_labels를 초과하면 낮은 점수 순으로 제거
    if len(final_labels_list) > max_labels:
        
        # 점수 기반으로 정렬: 점수가 높거나, 점수가 같으면 원본 레이블(selected_labels)에 있는 것을 우선 (추가된 부모가 제거될 확률 높임)
        def sort_key(cid):
            # (점수, 원본 레이블에 포함 여부)를 기준으로 정렬
            return (score_dict.get(cid, 0.0), cid in current_labels)

        final_labels_list.sort(key=sort_key, reverse=True)
        
        # 최대 레이블 개수에 맞게 자릅니다.
        final_labels_list = final_labels_list[:max_labels]
        
    return final_labels_list


# ============================================
# 5. Hybrid Taxo-Fusion Pipeline (통합 실행)
# ============================================

def run_integrated_hybrid_pipeline(
    train_corpus_path, test_corpus_path, strong_silver_path, final_silver_path,
    name2id, keyword_map, parent_map, child_map, G, final_submission_path,
    tfidf_ranking_map, 
    w_mlp_ratio=0.9, w_logreg_ratio=0.1, 
    alpha_mlp_taxo=0.2, 
    top_k=3,
    mlp_confidence_threshold=0.85,
    W_MODEL_TO_HEURISTIC=0.9,
    # 🚨 MLP 복잡도 변경을 위한 파라미터 추가
    MLP_HIDDEN_LAYERS=(256, 128)
):
    
    W_HEURISTIC = 1.0 - W_MODEL_TO_HEURISTIC # 0.1
    MIN_LABELS = 2
    MAX_LABELS = top_k # 3
    
    print(f"\n--- [START] Hybrid Pipeline (Final Fusion: Model * {W_MODEL_TO_HEURISTIC} + Heuristic * {W_HEURISTIC}) ---")

    # --- [STEP 1 & 2] 특징 추출 및 PCA 적용 ---
    print("\n[STEP 1 & 2] 특징 추출 및 PCA 적용")

    stopwords = load_stopwords(STOPWORD_PATH)
    pid2text = load_corpus(train_corpus_path, desc="Loading Train Corpus for Fusion")
    strong_labels = load_silver_labels(strong_silver_path)

    all_pids = list(pid2text.keys())
    all_texts = list(pid2text.values())
    
    X_BERT_ALL = get_bert_embeddings(all_texts, bert_tokenizer, bert_model, device, description="Encoding All Train Reviews (X_BERT)")
    
    strong_pids = [pid for pid in strong_labels.keys() if pid in pid2text]
    train_targets_strong = [strong_labels[pid] for pid in strong_pids]
    
    strong_indices = [all_pids.index(pid) for pid in strong_pids]
    X_BERT_STRONG = X_BERT_ALL[strong_indices]
    
    pca.fit(X_BERT_STRONG) 
    X_PCA_ALL = pca.transform(X_BERT_ALL) 
    X_PCA_STRONG = X_PCA_ALL[strong_indices]

    num_classes = max(name2id.values()) + 1 if name2id else 1
    mlb = MultiLabelBinarizer(classes=list(range(num_classes)))
    Y_strong = mlb.fit_transform(train_targets_strong)
    if hasattr(Y_strong, 'toarray'): Y_strong = Y_strong.toarray()
    
    
    # --- [STEP 2.5] TaxoClass 특징 및 키워드 특징 구축 (입력 특징) ---
    print("\n[STEP 2.5] 🧠 TaxoClass 특징 구축 및 Fusion Input 생성")
    
    id_to_name = {v: k for k, v in name2id.items()}
    id_to_description = load_class_descriptions(CLASS_DESCRIPTION_PATH, id_to_name, keyword_map)
    X_CLASS_FEATURES = build_class_features(id_to_description, bert_tokenizer, bert_model, device, G, num_classes, pca)
    
    S_Taxo_ALL = cosine_similarity(X_PCA_ALL, X_CLASS_FEATURES)
    S_Taxo_STRONG = S_Taxo_ALL[strong_indices] 
    S_Taxo_refined = apply_gnn_score_refinement(S_Taxo_ALL, G, num_classes)
    
    X_KEYWORD_ALL = build_keyword_features(all_pids, pid2text, keyword_map, tfidf_ranking_map, num_classes, stopwords)
    X_KEYWORD_STRONG = X_KEYWORD_ALL[strong_indices]

    # Fusion 특징 확장: X_PCA, S_Taxo, X_KEYWORD
    X_FUSION_ALL = np.concatenate([X_PCA_ALL, S_Taxo_ALL, X_KEYWORD_ALL], axis=1)
    X_FUSION_STRONG = np.concatenate([X_PCA_STRONG, S_Taxo_STRONG, X_KEYWORD_STRONG], axis=1)
    
    print(f"  --> Fusion Input Dimension: {X_FUSION_ALL.shape[1]} ({PCA_COMPONENTS} + {num_classes} + {num_classes})")


    # --- [STEP 3] 1차 앙상블 학습 및 S_MLP_DUMP 생성 ---
    print("\n[STEP 3] 1차 앙상블 학습 (Seed Model) - 확장된 Fusion Input 사용")

    # 🚨 MLP 복잡도 변경: (256, 128) -> (512, 256)
    base_clf_mlp = MLPClassifier(hidden_layer_sizes=MLP_HIDDEN_LAYERS, max_iter=500, alpha=5e-3, solver='adam', random_state=SEED, early_stopping=False)
    ovr_clf_mlp_1st = OneVsRestClassifier(base_clf_mlp, n_jobs=-1)
    ovr_clf_mlp_1st.fit(X_FUSION_STRONG, Y_strong)
    S_MLP_DUMP = ovr_clf_mlp_1st.predict_proba(X_FUSION_ALL)
    
    base_clf_logreg = LogisticRegression(solver='saga', C=1.0, max_iter=1000, n_jobs=-1, random_state=SEED, penalty='l2', tol=1e-3)
    ovr_clf_logreg_1st = OneVsRestClassifier(base_clf_logreg, n_jobs=-1)
    ovr_clf_logreg_1st.fit(X_FUSION_STRONG, Y_strong)
    S_LogReg = ovr_clf_logreg_1st.predict_proba(X_FUSION_ALL)

    # 🚨 앙상블 비율 변경: 0.9 -> 0.95
    S_Model_Ensemble = (w_mlp_ratio * S_MLP_DUMP + w_logreg_ratio * S_LogReg)
    print(f"  --> 1차 모델 앙상블 완료 (MLP:{w_mlp_ratio}, LogReg:{w_logreg_ratio}).")


    # --- [STEP 4] Pure Model Fusion Silver Label 생성 (휴리스틱 최종 융합 적용 및 구조적 보정) ---
    print("\n[STEP 4] 💡 Pure Model Fusion Silver Label 생성 (구조적 보정 적용)")

    # 🚨 Taxo 융합 비율 변경: alpha_mlp_taxo 사용
    S_Model_Ensemble_GNN = apply_gnn_score_refinement(S_Model_Ensemble, G, num_classes)
    S_MLP_Taxo_Hybrid = (alpha_mlp_taxo * S_Model_Ensemble_GNN) + ((1.0 - alpha_mlp_taxo) * S_Taxo_refined)
    S_MLP_Taxo_Hybrid_GNN = apply_gnn_score_refinement(S_MLP_Taxo_Hybrid, G, num_classes)

    CONFIDENCE_THRESHOLD = 0.15
    all_final_labels_map = {}
    
    for idx, (pid, text) in tqdm(enumerate(zip(all_pids, all_texts)), total=len(all_pids), desc="  Fusing Scores for Silver Label Generation"):
        final_scores = defaultdict(float)
        
        # 🚨 휴리스틱 점수 계산 및 융합
        tokens = clean_text_to_tokens(text, stopwords)
        heuristic_scores_raw = score_text_with_tfidf(
            pid, tokens, keyword_map, parent_map, child_map, tfidf_ranking_map, w_emotion=0.0
        )
        
        heuristic_scores = np.zeros(num_classes)
        for cid, score in heuristic_scores_raw.items():
             if cid < num_classes: heuristic_scores[cid] = score

        if np.max(heuristic_scores) > 0:
            heuristic_scores = heuristic_scores / np.max(heuristic_scores)


        s_model_taxo = S_MLP_Taxo_Hybrid_GNN[idx] 
        
        # 최종 융합: Model * 0.9 + Heuristic * 0.1
        final_hybrid_scores = (W_MODEL_TO_HEURISTIC * s_model_taxo) + (W_HEURISTIC * heuristic_scores)


        for cid in range(num_classes):
            final_scores[cid] = final_hybrid_scores[cid]

        if final_scores:
            sorted_scores = sorted(final_scores.items(), key=lambda x: -x[1])
            score_dict = {cid: sc for cid, sc in sorted_scores}
            
            # 1. 최소 레이블 제약 기반 선택
            selected_initial = refine_labels_with_min_constraint(
                sorted_scores, CONFIDENCE_THRESHOLD, min_labels=MIN_LABELS, max_labels=MAX_LABELS
            )
            
            # 2. 계층적 일관성 기반 보정 및 최대 레이블 수 재조정
            selected = enforce_hierarchical_consistency(
                selected_initial, parent_map, score_dict, MAX_LABELS
            )
            
        else: selected = [0]

        all_final_labels_map[pid] = selected

    # Silver Label CSV 저장
    print(f"  [SAVE] Pure Model Fusion Silver Labels 저장 중: {final_silver_path}")
    with open(final_silver_path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["pid", "labels"])
        for pid, labs in tqdm(all_final_labels_map.items(), total=len(all_final_labels_map), desc="Writing Final Silver CSV"):
            w.writerow([pid, ",".join(map(str, labs))])
    print("  --> Pure Model Fusion Silver Labels 저장 완료.")


    # --- [STEP 5] 최종 앙상블 학습 (MLP Confidence 필터링 적용 및 재학습) ---
    print("\n[STEP 5] 최종 앙상블 학습 (MLP Confidence 필터링 적용 및 재학습)")

    final_silver_train = load_silver_labels(final_silver_path)
    
    S_MLP_MAX_CONF = np.max(S_MLP_DUMP, axis=1) 
    
    filtered_indices = []
    all_pids_map = {pid: i for i, pid in enumerate(all_pids)}

    for pid, labs in final_silver_train.items():
        if not labs: continue
            
        idx = all_pids_map.get(pid)
        if idx is None: continue 
        
        if S_MLP_MAX_CONF[idx] >= mlp_confidence_threshold:
            filtered_indices.append(idx)
        

    X_final_train_filtered = X_FUSION_ALL[filtered_indices]
    training_pids_filtered = [all_pids[i] for i in filtered_indices]
    train_targets_final_filtered = [final_silver_train[pid] for pid in training_pids_filtered]
    
    Y_final_filtered = mlb.transform(train_targets_final_filtered)
    if hasattr(Y_final_filtered, 'toarray'): Y_final_filtered = Y_final_filtered.toarray()
    
    print(f"  --> 필터링된 최종 학습 데이터 크기: {X_final_train_filtered.shape[0]} / {X_FUSION_ALL.shape[0]}")
    
    if X_final_train_filtered.shape[0] == 0:
        print("  ❌ WARNING: 필터링 후 유효한 학습 샘플이 없어 예측을 건너뜁니다.")
        return 
        
    # 🚨 MLP 복잡도 변경 적용
    base_clf_mlp_final = MLPClassifier(hidden_layer_sizes=MLP_HIDDEN_LAYERS, max_iter=1000, alpha=5e-3, solver='adam', random_state=SEED, early_stopping=False)
    ovr_clf_mlp_final = OneVsRestClassifier(base_clf_mlp_final, n_jobs=-1)
    ovr_clf_mlp_final.fit(X_final_train_filtered, Y_final_filtered) 
    
    base_clf_logreg_final = LogisticRegression(solver='saga', C=1.0, max_iter=3000, n_jobs=-1, random_state=SEED, penalty='l2', tol=1e-3)
    ovr_clf_logreg_final = OneVsRestClassifier(base_clf_logreg_final, n_jobs=-1)
    ovr_clf_logreg_final.fit(X_final_train_filtered, Y_final_filtered)
    
    print("  --> 최종 MLP/LogReg 앙상블 학습 완료 (MLP Confidence 필터링 기반).")


    # --- [STEP 6] 최종 예측 및 파일 저장 (휴리스틱 최종 융합 적용) ---
    print("\n[STEP 6] 최종 예측 및 파일 저장 (휴리스틱 최종 융합 적용)")

    test_corpus = load_corpus(test_corpus_path, desc="Loading Test Corpus for Prediction")
    test_pids = list(test_corpus.keys())
    test_texts = [test_corpus[pid] for pid in test_pids]
    
    X_BERT_TEST = get_bert_embeddings(test_texts, bert_tokenizer, bert_model, device, description="Encoding Test Reviews (X_BERT)")
    X_PCA_TEST = pca.transform(X_BERT_TEST) 
    S_Taxo_Test = cosine_similarity(X_PCA_TEST, X_CLASS_FEATURES) 
    
    X_KEYWORD_TEST = build_keyword_features(test_pids, test_corpus, keyword_map, tfidf_ranking_map, num_classes, stopwords)
    
    X_FUSION_TEST = np.concatenate([X_PCA_TEST, S_Taxo_Test, X_KEYWORD_TEST], axis=1) 
    
    mlp_probas = ovr_clf_mlp_final.predict_proba(X_FUSION_TEST) 
    logreg_probas = ovr_clf_logreg_final.predict_proba(X_FUSION_TEST) 
    
    # 🚨 앙상블 비율 변경 적용
    ensemble_probas = (w_mlp_ratio * mlp_probas + w_logreg_ratio * logreg_probas)
    
    S_Taxo_Test_refined = apply_gnn_score_refinement(S_Taxo_Test, G, num_classes)
    
    # 1차 융합 (Model + Taxo)
    decision_scores_hybrid = (alpha_mlp_taxo * ensemble_probas) + ((1.0 - alpha_mlp_taxo) * S_Taxo_Test_refined)
    
    # 🚨 휴리스틱 점수 계산 및 최종 융합 (W_HEURISTIC = 0.1)
    final_hybrid_probas = decision_scores_hybrid.copy()
    W_HEURISTIC_FINAL = 1.0 - W_MODEL_TO_HEURISTIC # 0.1
    
    for i, pid in tqdm(enumerate(test_pids), total=len(test_pids), desc="Final Heuristic Blending"):
        tokens = clean_text_to_tokens(test_corpus.get(pid, ""), stopwords)
        heuristic_scores_raw = score_text_with_tfidf(
            pid, tokens, keyword_map, parent_map, child_map, tfidf_ranking_map, w_emotion=0.0
        )
        
        heuristic_scores = np.zeros(num_classes)
        for cid, score in heuristic_scores_raw.items():
            if cid < num_classes: heuristic_scores[cid] = score

        if np.max(heuristic_scores) > 0:
            heuristic_scores = heuristic_scores / np.max(heuristic_scores)
            
        # 최종 융합: Model * 0.9 + Heuristic * 0.1
        final_hybrid_probas[i] = (W_MODEL_TO_HEURISTIC * decision_scores_hybrid[i]) + (W_HEURISTIC_FINAL * heuristic_scores)


    decision_scores_refined = apply_gnn_score_refinement(final_hybrid_probas, G, num_classes)
    
    pred_labels = []
    for i in tqdm(range(X_PCA_TEST.shape[0]), desc="  Selecting Top-3 and Saving"):
        row = decision_scores_refined[i]
        sorted_scores = sorted(enumerate(row), key=lambda x: -x[1])
        score_dict = {cid: sc for cid, sc in sorted_scores}
        
        # 1. 최소 레이블 제약 기반 선택
        selected_initial = refine_labels_with_min_constraint(
            sorted_scores, CONFIDENCE_THRESHOLD, min_labels=MIN_LABELS, max_labels=MAX_LABELS
        )
        
        # 2. 계층적 일관성 기반 보정 및 최대 레이블 수 재조정
        selected = enforce_hierarchical_consistency(
            selected_initial, parent_map, score_dict, MAX_LABELS
        )
        
        pred_labels.append(selected)

    print(f"\n[SAVE] 최종 제출 파일 저장 중: {final_submission_path}")
    with open(final_submission_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f); writer.writerow(["id", "label"])
        for pid, labs in tqdm(zip(test_pids, pred_labels), total=len(test_pids), desc="Writing Submission CSV"):
            writer.writerow([pid, ",".join(map(str, labs))])
    print("Submission saved ✅")


if __name__ == "__main__":
    
    # 🚨🚨🚨 하이퍼파라미터 설정 (여기서 조절하세요) 🚨🚨🚨
    # 🚨 1. LogReg 역할 축소 (MLP에 집중)
    W_MLP_RATIO = 0.75       #95
    W_LOGREG_RATIO = 0.25     #5
    
    # 🚨 2. Taxo 융합 강화 (모델보다 Taxo 지식에 더 많은 영향력)
    ALPHA_MLP_TAXO = 0.25     
    
    MLP_CONFIDENCE_THRESHOLD = 0.85 
    W_MODEL_TO_HEURISTIC_FINAL = 0.9 

    # 🚨 3. MLP 구조 복잡도 증가
    MLP_HIDDEN_LAYERS = (256, 128) 


    print("=== START: Integrated Hybrid Pipeline (Arch Upgraded & LogReg Reduced) ===")

    # 1. Resources Load
    name2id = load_class_names(CLASS_PATH)
    keyword_map = load_keywords(KEYWORD_PATH, name2id)
    parent_map, child_map, G = load_hierarchy(HIER_PATH, name2id) 
    
    TFIDF_RANKING_MAP = load_tfidf_ranking(TFIDF_RANKING_PATH)
    print(f"[SETUP] Loaded TFIDF Ranking Map for {len(TFIDF_RANKING_MAP)} documents.")

    # 2. Run Integrated Pipeline
    run_integrated_hybrid_pipeline(
        TRAIN_CORPUS, TEST_CORPUS, STRONG_SILVER_PATH, TRAIN_SILVER_PATH,
        name2id, keyword_map, parent_map, child_map, G, FINAL_SUBMISSION,
        TFIDF_RANKING_MAP, 
        w_mlp_ratio=W_MLP_RATIO, w_logreg_ratio=W_LOGREG_RATIO, 
        alpha_mlp_taxo=ALPHA_MLP_TAXO,
        top_k=3,
        mlp_confidence_threshold=MLP_CONFIDENCE_THRESHOLD,
        W_MODEL_TO_HEURISTIC=W_MODEL_TO_HEURISTIC_FINAL, # 0.9 전달
        MLP_HIDDEN_LAYERS=MLP_HIDDEN_LAYERS # 새 파라미터 전달
    )
    ##이걸로 할지두